In [542]:
import numpy as np
import pandas as pd

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

import pickle

import joblib
import re
import string

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import BayesianRidge



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\irgit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [515]:
# Подготовка тувинского датасета
shyn = pd.read_csv(r'e:\WORK\Tu_RU\datasets\tu\tuva_text.csv')
gov = pd.read_csv(r'e:\WORK\Tu_RU\datasets\tu\govtuva.csv')

shyn.columns = ['id','text']
gov.columns = ['id','text']

data = pd.concat([shyn,gov], ignore_index=True)
del data['id']
data.dropna
df_tu = data

df_tu['type'] = 1

In [516]:
# Подготовка русского датасета
folder = r'e:\WORK\Tu_RU\datasets\ru\news\KP\texts'
text = []
file_list = glob.glob(folder + "/*.txt")
for x in file_list:
    with open(x, encoding="utf8") as f:        
        lines = f.readlines()
        text.append(' '.join(lines))
df_ru = pd.DataFrame(text)
df_ru['type'] = 0
df_ru.columns = ['text','type']
df_ru = df_ru[0:9000]
            

In [526]:
df = pd.concat([df_tu[0:500],df_ru[0:500]], ignore_index=True)

In [527]:
# Функция отчистки текста
def text_preroccesor(text):
    text = str(text).lower()
    tokens = [x for x in text.split() if x not in russian_stopwords and x.isalpha()]
    return ' '.join(tokens)

In [528]:
# Ошищаем датафрейм от мусора text_preroccesor(text)
df['text'] = df['text'].apply(lambda x: text_preroccesor(x))

In [529]:
# Разбиваем выборку на тренировачную и тестовую выборку
df_train, df_test = train_test_split(df, test_size=0.20, stratify=df.type)

# Делаем векторизацию
vec = CountVectorizer(
    ngram_range=(1, 3), 
    stop_words=None,
)

# Применяем метод трансформеров
X_train = vec.fit_transform(df_train.text)
X_test = vec.transform(df_test.text)

y_train = df_train.type
y_test = df_test.type

In [530]:
# Обучаем модель clf на базе MultinomialNB 
# https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

clf = MultinomialNB()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)

print(classification_report(y_test, preds))
print("Test score: {0:.2f} %".format(100 * score))

# Инфа по плотности слова в векторе (с маленькой буквы)
print(f'Плотности слова в векторе', vec.vocabulary_.get(u'кызыл'))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       100

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Test score: 99.98 %
Плотности слова в векторе 112508


In [531]:
# Тесттируем на Датафрейме
def get_predict(text):
    data = [text]   
    x_new = vec.transform(data)
    preds = clf.predict(x_new)
    return preds[0]

df_test = df_train[40:50]
df_test['predict'] = df_test['text'].apply(lambda x: get_predict(x))
df_test

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


,text,type,predict
288,чылдың июль хүндүлүг солуннуң чагыдып ооң идег...,1,1
175,алефтина суурда кожуун чергелиг культура килди...,1,1
240,россия биле тываның демнежилгезиниң чылынга чу...,1,1
78,тыва республиканың дээди хуралында тыва респуб...,1,1
703,съемки программы первого канала завершились ал...,0,0
201,чоокта чаа республиканың тыва спортчулары ново...,1,1
689,псковский государственный университет совместн...,0,0
566,псковские фехтовальщики поучаствовали турнире ...,0,0
475,марттың сураглыг ыраажы чаңгыс намчылактың төр...,1,1
162,тыва парлалганың хүндүлүг сеткиливис ханызында...,1,1


In [540]:
# Тестируем на тексте
text_in = """
Эрткен чылдың түңнелдери-биле алырга, Тывада туберкулезтан аарыыр чорук эвежээни демдеглеттинген. Фтизиатрлар ниитилелиниң хуралында, республиканың фтизиатрия албанының дыңнаткан сан-чурагайлары-биле алырга, 2022 чылда 100 муң кижиге онааштыр 113,5 кижи аарып, 2021 чылдан 8 хуу эвээжээни айыттынган. Бир дугаар илереттинген аарыг кижилер саны 11,6 хуу кызырылган ( 45,3 хуудан 33,7 хууже).
Туберкулезтан аарыыр болгаш өлүп-хораар чорукту элээн эвээжедир дизе, шупту субъектилерниң талазындан кады ажылдажылганы база ведомстволар аразында удур-дедир харылзаалыг ажылды күштелдирер херек деп, специалистер санаан. Ол ышкаш чон-биле санитарлыг-чырыдыышкын ажылын идепкейжидери чугула деп, эмчилер демдеглээн.
Халдавырлыг аарыглар санын эвээжедири база дезинфекцияны чорудары чугула черни ойнаарын демдеглээн. Регион чазааның деткимчези-биле эрткен чылын тус черниң фтизатрия албаны көжүп чоруур чаа ийи дезинфекция камераларын садып алганы, эпидемияның үнүп кээп болур черлерин каш катап эвээжеткен. Дезкамераларның бирээзин доктаамал ажыглаар кылдыр Барыын-Хемчикче чоруткан. Оон Бай-Тайга, Чөөн-Хемчик, Өвүр база Мөңгүн-Тайга кожууннарже үнүүшкүннерже үнер. Дезинфекция дериг-херекселдерин немей тургусканы ажылдың шынарынга салдарны чедирер деп, специалистер санаан.
Россияның Президентизи Владимир Путинниң регионга тускайжыттынган чаа эмчи объектизин – Тывага туберкулезка удур амгы үениң диспансер тудуун деткээни-биле, бо чоокку чылдарда ол тудуг туттунарын сагындыраал. Амгы үеде объектиниң төлевилел-смета документациязын белеткеп турар. Тываның Баштыңы Владислав Ховалыг туберкулезка удур демисел мурнады боттандырар ажылдарның бирээзи болганда, диспансер тудуунга хамааржыр шупту айтырыгларны бодунуң контролюнда тудуп турарын чугаалаан.
Тываның Баштыңы бодунуң Айыткалында сонуургалдыг ведомстволар-биле кады туберкулезка удур демисежир үр хуусаалыг регион программазын ажылдып кылгаш, “орук картазының” хемчеглерин база сорулгалыг көргүзүглерин чада аайы-биле чедип алыр даалганы республиканың Кадык камгалал яамызынга берген турган.
"""
x_new = vec.transform([text_in])
preds = clf.predict(x_new)

if preds[0] == 1:
    print('Текст на тувинском языке')
else:
    print('Текст на русском языке')

Текст на тувинском языке


In [533]:
# Сохранить модель
model_file = r'e:\WORK\Tu_RU\datasets\turu_model.pkl'

with open(model_file, 'wb') as file: 
    #pickle.dump(clf, file)
    pickle.dump((vec, clf), file)